In [1]:
import numpy as np
import re
import nltk
import pandas as pd
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /home/qubit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing punctuation
    - Lowering text
    """
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [3]:
#Importing DataFrame
raw_dataframe = pd.read_excel("Datasets _ NLP based tagging solution078466f.xlsx",sheet_name = None)

training_data = raw_dataframe['Training Data ']
tag_frame = raw_dataframe['Keywords for tagging']

Bus_Exp_Tags = tag_frame['Keywords for Business Exception']
Bus_Exp_Tags = Bus_Exp_Tags.iloc[0:8]
Sof_Exp_Tags = tag_frame['Keywords for System Exception']


In [4]:
#Preprocessing Data
Bus_Exp_Tags.apply(clean_text)
Sof_Exp_Tags.apply(clean_text)
Bus_Exp_Tags = Bus_Exp_Tags.unique()
Sof_Exp_Tags = Sof_Exp_Tags.unique()

Train_data = training_data['Exception (input)'].apply(clean_text)
Actual_Class = training_data['Exception Category (ouput)']

In [5]:
#Removing StopWords
Words=[]
for line in Train_data:
    text_tokens = line.split()
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    Words.append(tokens_without_sw)

Words_BE = []
Words_SE = []

for line in Bus_Exp_Tags:
    text_tokens = line.lower().split()
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    Words_BE += tokens_without_sw

for line in Sof_Exp_Tags:
    text_tokens = line.lower().split()
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    Words_SE += tokens_without_sw
    

In [6]:
#Assigning weights to tags inversely proportional to their frequency
ALL_WORDS = {}
BE_WORDS = {}
SE_WORDS = {}

for line in Words:
    for tok in line:
        if tok in ALL_WORDS:
            ALL_WORDS[tok] += 1
        else :
            ALL_WORDS[tok] = 1


for word in Words_BE:
    if word in ALL_WORDS :
        BE_WORDS[word] = ALL_WORDS[word]
    else:
        BE_WORDS[word] = 0
        
for word in Words_SE:
    if word in ALL_WORDS :
        SE_WORDS[word] = ALL_WORDS[word]
    else:
        SE_WORDS[word] = 0
max_count_BE = max(BE_WORDS.values())
max_count_SE = max(SE_WORDS.values())

for word in BE_WORDS:
    BE_WORDS[word]=max_count_BE-BE_WORDS[word]
    
for word in SE_WORDS:
    SE_WORDS[word]=max_count_SE-SE_WORDS[word]+1


In [7]:
#Rule Based Classification
Pred_Class = []
i=0
for line in Words:
    count_BE=0
    count_SE=0

    
    for keyword in Words_BE:
        if keyword in line:
#             count_BE+=1
            count_BE+=BE_WORDS[keyword]
            
    for keyword in Words_SE:
        if keyword in line:
#             count_SE+=1
            count_SE+=SE_WORDS[keyword]
    
    if count_BE >= count_SE:
        Pred_Class.append("Business Exception")
    else:
        Pred_Class.append("System Exception")

In [8]:
#Accuracy
count = 0
for i in range(0,len(Pred_Class)):
    if Pred_Class[i] == Actual_Class[i]:
        count+=1
print(count*100.0/len(Pred_Class))

93.61702127659575
